Create:


1. Function using keyword args
2. Func using non keyword args
3. Func with local and global scoped variables
4. Generate a receipt function

In [ ]:
#!pip install pdfkit

## Installs the library dependencies for wkhtmlt tox needed for the pdfkit library
# !wget http://archive.ubuntu.com/ubuntu/pool/main/o/openssl/libssl1.1_1.1.1f-1ubuntu2_amd64.deb
# !sudo dpkg -i libssl1.1_1.1.1f-1ubuntu2_amd64.deb

## Checks the OS Details and Version
# !cat /etc/os-release
# !uname -m

# !wget https://github.com/wkhtmltopdf/packaging/releases/download/0.12.6-1/wkhtmltox_0.12.6-1.bionic_amd64.deb
# !cp wkhtmltox_0.12.6-1.bionic_amd64.deb /usr/bin
# !sudo apt install /usr/bin/wkhtmltox_0.12.6-1.bionic_amd64.deb

In [ ]:
import pdfkit
from datetime import datetime

#List of stocks in shop
stocks = {
        "shoes": [
            {'unit_price':3000},
            {'quantity': 30}
        ],
        "jerseys": [
            {'unit_price':1000},
            {'quantity': 10}
        ],
        "vests": [
            {'unit_price':100},
            {'quantity': 1000}
        ],
        "socks": [
            {'unit_price':50},
            {'quantity': 1000}
        ],
        "bags": [
            {'unit_price':3000},
            {'quantity': 5}
        ]
}

In [ ]:
list_of_items = []
selected_items = []

def display_catalogue(**kwargs) -> int:
  '''
    Gets the number of items in our catalogue -> items_in_stock
    Iterates through the dict{} and adds the keys(item names) in list_of_items
    Prints the item names together with their stock prices in ascending order
  '''

  items_in_stock = 0

  for x, y in kwargs.items():
    list_of_items.append(x) #Add to list of items for user comparison
    items_in_stock += 1
    properties = y[0]
    unit_price = list(map(properties.get, properties)) #todo: find a way to optimise for speed
    print(f'{items_in_stock}. {x.capitalize()} @ Kshs. {unit_price[0]}')
  return items_in_stock

def get_customer_item_quantity(item) -> int:
  return int(input(f'How many {item} do you want? \n'))

item_amount = lambda x , y: x * y

def calculate_basket_cost(prompt, item_count) -> int:
  total_cost = 0

  try:
    if prompt > item_count + 1:
      print('Kindly enter an item that is in stock')
      calculate_basket_cost(int(input('Enter an item')))
    elif prompt == item_count + 1:
      return
    else:

      selected_item = list_of_items[prompt - 1]

      #todo: check if the selected item is in stock
        #if in stock -> reduce the total items in stock
      quantity = get_customer_item_quantity(selected_item)

      #calculate the cost of the selected item
      unit_price = stocks[selected_item][0]['unit_price']

      #add the result to a total amount
      cost = total_cost + item_amount(unit_price, quantity)

      #add the selected items and cost to our selected_items list
      selected_items.append({
          "item": selected_item,
          "price": cost
      })

      return cost
  except ValueError:
    print('Only digit items are allowed!')
    calculate_basket_cost(int(input('Enter an item')))

def generate_customer_bill(item_count):
  exit_choice = item_count + 1
  shop_exit = 'Thanks for shopping with us. Come again soon!!!'

  selection_prompt = int(input(f'What would you like to purchase from the list above? To select enter the preceeding digit or enter {exit_choice} to Exit\n'))

  if selection_prompt == exit_choice:
    return shop_exit
  else:
    current_cost = calculate_basket_cost(selection_prompt, item_count)

    cost_of_items = add_to_basket(item_count, current_cost)

    if cost_of_items > 0:
      print(f'Your total bill is: {cost_of_items}')
      return cost_of_items
    else:
      return shop_exit


def add_to_basket(*args):
  total_cost = args[1]
  while True:
    proceed_prompt = int(input(f'Your current bill is Kshs. {total_cost}. \n Do you want to add to your basket? Input 1 to proceed, 2 to not add to your basket, 3 to exit \n'))

    if proceed_prompt == 1:
      selection_prompt = int(input('Add to basket: '))
      current_cost = calculate_basket_cost(selection_prompt, args[0])
      total_cost += current_cost
    elif proceed_prompt == 2:
      break
    else:
      total_cost = 0
      #Exit
      break;

  return total_cost

def transform_customer_selected_items(data) -> str:
  i = ''
  for x in data:
    name = x['item']
    price = x['price']

    i += f'<p>{name.capitalize()} {price} </p>'

  return i


def generate_customer_receipt(total_cost):
  header = f'\
  <main><body>\
  <div><h2>Receipt (Norman\'s Shop)</h2></div>\
  <div><p>Address: P.O Box 254 GenZ Station, Nairobi, Kenya</p></div>\
  <div>+254.712.345.678</div>\
  {"-"*150}\
  <div>Date: {datetime.now().strftime("%d/%m/%Y %H:%M")}</div>\
  {"-"*150}\
  <div>\
  {transform_customer_selected_items(selected_items)}\
  </div>\
  {"-"*150}\
  <footer><h3>Total amount {total_cost}</h3></footer>\
  </body></main>\
  '

  return pdfkit.from_string(header, 'receipt.pdf')


In [ ]:
print('Welcome to Norman\'s Shop - guranteed best prices in the market. Here\'s a list of the items in our catalogue \n')
print('-'*150)

num_of_items = display_catalogue(**stocks)
print('-'*150)
customer_bill = generate_customer_bill(num_of_items)

if isinstance(customer_bill, int):
  print('-'*150)
  #todo: Add prompt to ask user how they'd like to pay
  print('Generating receipt ...')
  receipt = generate_customer_receipt(customer_bill)

  cashier_response = lambda x: 'Enjoy your items and thanks for shopping with us!' if x == True else 'Error generating receipt!'

  print(cashier_response(receipt))
else:
  print(customer_bill)

Welcome to Norman's Shop - guranteed best prices in the market. Here's a list of the items in our catalogue 

------------------------------------------------------------------------------------------------------------------------------------------------------
1. Shoes @ Kshs. 3000
2. Jerseys @ Kshs. 1000
3. Vests @ Kshs. 100
4. Socks @ Kshs. 50
5. Bags @ Kshs. 3000
------------------------------------------------------------------------------------------------------------------------------------------------------
What would you like to purchase from the list above? To select enter the preceeding digit or enter 6 to Exit
1
How many shoes do you want? 
3
Your current bill is Kshs. 9000. 
 Do you want to add to your basket? Input 1 to proceed, 2 to not add to your basket, 3 to exit 
1
Add to basket: 4
How many socks do you want? 
7
Your current bill is Kshs. 9350. 
 Do you want to add to your basket? Input 1 to proceed, 2 to not add to your basket, 3 to exit 
2
Your total bill is: 9350
-